In [4]:

!pip install llama-index-vector-stores-postgres


  Obtaining dependency information for llama-index-vector-stores-postgres from https://files.pythonhosted.org/packages/ae/08/6bf8d2a046437c891dfa2a37836eec341ddf1ff160abfdf3cf4fb3d90e5f/llama_index_vector_stores_postgres-0.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for asyncpg<0.30.0,>=0.29.0 from https://files.pythonhosted.org/packages/99/38/0bfb00e9b828513bd759174860fd2b1c5e36d0b33985c90ff4ed6f96814c/asyncpg-0.29.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for pgvector<0.3.0,>=0.2.4 from https://files.pythonhosted.org/packages/29/bb/4686b1090a7c68fa367e981130a074dc6c1236571d914ffa6e05c882b59d/pgvector-0.2.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for psycopg2-binary<3.0.0,>=2.9.9 from https://files.pythonhosted.org/packages/19/57/9f172b900795ea37246c78b5f52e00f4779984370855b3e161600156906d/psycopg2_binary-2.9.9-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata

In [1]:
import pathlib

images_path = pathlib.Path("./resources")

image_filenames = list(images_path.iterdir())


In [3]:
from llama_index.core import SimpleDirectoryReader

image_documents = SimpleDirectoryReader(input_files=image_filenames).load_data()


In [6]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()
connection_string = os.environ['POSTGRES_CONNECTION_URL']
db_name = "vector_db"

conn = psycopg2.connect(connection_string)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

In [8]:
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.postgres import PGVectorStore

from sqlalchemy import make_url
url = make_url(connection_string)

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=url.host,
    password=url.password,
    port=url.port,
    user=url.username,
    table_name="paul_graham_essay",
    embed_dim=1536,  # openai embedding dimension
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    image_documents, storage_context=storage_context, show_progress=True
)
query_engine = index.as_query_engine()

Parsing nodes:   0%|          | 0/12 [00:00<?, ?it/s]

ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******